In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/data_0625')

Mounted at /content/drive


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings(action='ignore')


#### Data loading

In [12]:
train=pd.read_csv('./0625train.csv')
train_labels=pd.read_csv('./train_labels.csv')
test=pd.read_csv('./0625test.csv')
submission=pd.read_csv('./sample_submission.csv')

pd.options.display.max_columns=50

In [ ]:
# test.rename(columns = {'Time' : 'time'}, inplace = True)
# test.dropna(axis=0,inplace=True)
# test

In [13]:
train.drop('exercise',axis=1,inplace=True)
test.drop('exercise',axis=1,inplace=True)

#### Feature engneering

#####  가속도, 자이로, (자이로-가속도) 센서값을 에너지로 표현

In [14]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/2)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/2)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/2)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/2)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/2)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/2)

###### id별 데이터는 0.02초마다 측정된 값들이기 때문에 이전 시간 대비 변화량 적용

In [15]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [16]:
train_dt=[]
# tqdm => 진행표시바 나오게 함
# np.unique => 고유한 원소의 집합 
for i in tqdm(train['id'].unique()): 
    # 한 아이디에 600rows가 있음
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)


100%|██████████| 468/468 [00:06<00:00, 76.20it/s]


In [17]:
train_dt[0]

id                       Time     acc_x     acc_y     acc_z    gy_x  \
0     0  2021-06-24 14:28:58:76551  0.785236  0.131117  0.591857   -9.73   
1     0  2021-06-24 14:28:59:10611  0.802572  0.121106  0.571103   -4.83   
2     0  2021-06-24 14:28:59:12814  0.822838  0.095713  0.580137  -28.91   
3     0   2021-06-24 14:28:59:1502  0.833337  0.073982  0.598206  -23.66   
4     0  2021-06-24 14:28:59:16824  0.841151  0.055181  0.596496  -17.92   
..   ..                        ...       ...       ...       ...     ...   
595   0   2021-06-24 14:29:11:0377  1.177366  0.268826  0.269559  383.95   
596   0  2021-06-24 14:29:11:05793  1.020612  0.356726  0.320589  367.92   
597   0  2021-06-24 14:29:11:07833  0.802816  0.417523  0.362342  319.13   
598   0  2021-06-24 14:29:11:09777  0.603089  0.354773  0.474658  224.28   
599   0  2021-06-24 14:29:11:11815  0.497854  0.133070  0.634342  127.61   

       gy_y   gy_z  acc_Energy   gy_Energy  gy_acc_Energy   acc_x_dt  \
0     -4.20  -5.67    0.992010   12.019227      12.982282   0.000000   
1     -2.66  -3.57    0.992445    6.568820       7.523906   0.866788   
2     -5.60   5.88    1.011327   30.028695      30.733879   1.013287   
3     -4.90   2.31    1.028481   24.272241      25.051832   0.524954   
4     -4.76   0.35    1.032660   18.544716      19.370790   0.390673   
..      ...    ...         ...         ...            ...        ...   
595  110.81  10.29    1.237385  399.752852     398.540650   1.208622   
596  138.04  19.95    1.127688  393.469402     392.373749  -7.837719   
597  156.17  26.04    0.974747  356.245852     355.317284 -10.889781   
598  164.99  14.35    0.845506  278.799464     278.080283  -9.986374   
599  145.74   4.97    0.817286  193.776058     193.333064  -5.261768   

      acc_y_dt  acc_z_dt      gy_x_dt  gy_y_dt  gy_z_dt  acc_Energy_dt  \
0     0.000000  0.000000     0.000000      0.0      0.0       0.000000   
1    -0.500539 -1.037703   245.000025     77.0    105.0       0.021788   
2    -1.269661  0.451708 -1204.000000   -147.0    472.5       0.944092   
3    -1.086537  0.903410   262.500000     35.0   -178.5       0.857709   
4    -0.940037 -0.085458   287.000000      7.0    -98.0       0.208944   
..         ...       ...          ...      ...      ...            ...   
595   5.823348  0.500540  1368.500000    987.0    315.0       2.278242   
596   4.394980  2.551531  -801.500000   1361.5    483.0      -5.484849   
597   3.039864  2.087615 -2439.500000    906.5    304.5      -7.647052   
598  -3.137529  5.615807 -4742.500000    441.0   -584.5      -6.462054   
599 -11.085118  7.984213 -4833.500000   -962.5   -469.0      -1.411006   

     gy_Energy_dt  gy_acc_Energy_dt  
0        0.000000          0.000000  
1     -272.520319       -272.918801  
2     1172.993716       1160.498682  
3     -287.822663       -284.102358  
4     -286.376256       -284.052117  
..            ...               ...  
595   1585.267659       1582.540927  
596   -314.172489       -308.345046  
597  -1861.177489      -1852.823255  
598  -3872.319435      -3861.850029  
599  -4251.170296      -4237.360939  

[600 rows x 20 columns]

In [18]:
test_dt=[]
# train과 같은 방식으로 진행
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|██████████| 44/44 [00:00<00:00, 77.36it/s]


##### 가속도, 자이로 센서값들을 푸리에 변환

In [19]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    # fft => Fast Fourier transforms
    # 매개변수로 받은 t_signal을 푸리에 변환 시킨다
    complex_f_signal= fftpack.fft(t_signal)
     
    # 주파수 해석의 목적은 각 주파수 별로 크기 (magnitude)가 어느 정도인지 알아내는 것
    # 따라서 절대값 함수 abs()를 이용해서 크기를 연산할 수 있음
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [20]:
train=pd.concat(train_dt)

In [ ]:
# train

In [21]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    # acc값과 gy 값만 사용
    for i in train.columns[2:8]:
        # 위에서 선언한 함수 사용해서 푸리에 변환
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████| 468/468 [00:00<00:00, 488.92it/s]


In [ ]:
# train

In [22]:
test=pd.concat(test_dt)

In [23]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|██████████| 44/44 [00:00<00:00, 589.96it/s]


##### Standard scaling 적용


In [24]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [25]:
test_s

id                       Time       acc_x       acc_y       acc_z  \
0       0  2021-06-21 16:49:00:79942  405.982400  148.941731  182.118464   
1       0   2021-06-21 16:49:00:8194   42.481900   48.839350  119.699179   
2       0  2021-06-21 16:49:00:83957   51.397819   37.820433   94.558268   
3       0  2021-06-21 16:49:00:86032   33.653027   23.112934   78.324091   
4       0  2021-06-21 16:49:00:87975   38.771849   43.634004   35.550969   
...    ..                        ...         ...         ...         ...   
26395  43  2021-06-21 17:10:48:51296   23.661999   14.415446   34.209396   
26396  43    2021-06-21 17:10:48:533   42.404959   17.026743   46.229746   
26397  43  2021-06-21 17:10:48:55273  140.108815   31.551633  120.035712   
26398  43  2021-06-21 17:10:48:57292   18.348290   24.320396    8.005155   
26399  43   2021-06-21 17:10:48:5926   14.945484   12.702734   10.461576   

               gy_x         gy_y          gy_z  acc_Energy   gy_Energy  \
0       6060.460009   595.630020   1424.570011    1.016275   53.103663   
1       8183.130793  3339.621481   3047.685915    1.017473    6.768678   
2       2984.200237  4972.487780   1962.874567    1.030642    4.846710   
3       5891.901606  3306.428620   9048.417451    1.032922    4.154841   
4       8988.799416  1016.684891  10059.624137    1.041546    6.590789   
...             ...          ...           ...         ...         ...   
26395  13797.364583  5349.453313   6651.688424    1.075159  284.651899   
26396   5739.550817  1152.378651   2400.115461    1.152994  245.570306   
26397  38813.986546  8546.205289  18554.249952    1.250243  210.973509   
26398  11863.024830  3130.582839   3476.174360    1.221321  216.977689   
26399   4301.383505  1778.048262   3130.485090    1.255238  245.509520   

       gy_acc_Energy  acc_x_dt   acc_y_dt  acc_z_dt  gy_x_dt     gy_y_dt  \
0          53.334841  0.000000   0.000000  0.000000      0.0     0.00000   
1           7.616737  0.732495   0.744706  0.622622    101.5 -1984.49999   
2           5.690957 -0.036622  -1.733577  0.012208     59.5   -73.50001   
3           4.834034  0.109878   0.463916 -0.537165     87.5   -14.00000   
4           6.405640  0.256372  -1.049913  0.537165    269.5     7.00000   
...              ...       ...        ...       ...      ...         ...   
26395     283.685559 -3.870020   7.740048  2.685823  -3811.5   189.00000   
26396     244.486300  2.075403  11.378116  4.468228  -2355.5    66.50000   
26397     209.855172  4.028735   2.954404  1.880077  -2446.5  -196.00000   
26398     215.875661 -3.076490  -7.300551  6.409347    532.0    98.00000   
26399     244.318036  1.452786   2.832320  0.256371   1767.5   -63.00000   

       gy_z_dt  acc_Energy_dt  gy_Energy_dt  gy_acc_Energy_dt  
0          0.0       0.000000      0.000000          0.000000  
1       1400.0       0.059892  -2316.749237      -2285.905190  
2        -28.0       0.658478    -96.098390        -96.288999  
3         28.0       0.113973    -34.593484        -42.846149  
4         24.5       0.431231    121.797424         78.580297  
...        ...            ...           ...               ...  
26395   -735.0      -4.043404  -3528.319445      -3525.840168  
26396   -164.5       3.891759  -1954.079656      -1959.962959  
26397    255.5       4.862430  -1729.839885      -1731.556410  
26398   -192.5      -1.446071    300.209032        301.024449  
26399    287.0       1.695823   1426.591550       1422.118730  

[26400 rows x 20 columns]

In [26]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Standard => 기본 스케일. 평균과 표준편차 사용
# 평균을 제거하고 데이터를 단위 분산으로 조정
# 이상치가 있다면 평균과 표준편차에 영향을 미쳐 변환된 데이터의 확산은 매우 달라지게 된다
# 따라서 이상치가 있는 경우 균형 잡힌 척도를 보장할 수 없다
scaler = StandardScaler()


# 변환만을 생각한다면 fit() , transform()을 함께 사용하지 않고 
# transform()만 사용하면 될텐데 두개 메소드를 함께 사용하는 이유가 있다

# 학습데이터 세트에서 변환을 위한 기반 설정(예를 들어 학습 데이터 세트의 최대값/최소값등)을 
# 먼저 fit()을 통해서 설정한 뒤에 이를 기반으로 학습 데이터의 transform()을 수행하되 
# 학습 데이터에서 설정된 변환을 위한 기반 설정을 그대로 테스트 데이터에도 적용하기 위해서이다
# 참고 https://subinium.github.io/MLwithPython-3-3/
train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [27]:
test_sc

id                       Time      acc_x      acc_y      acc_z  \
0       0  2021-06-21 16:49:00:79942  21.396602  11.655448  13.590342   
1       0   2021-06-21 16:49:00:8194   1.963795   3.540409   8.758492   
2       0  2021-06-21 16:49:00:83957   2.440442   2.647134   6.812344   
3       0  2021-06-21 16:49:00:86032   1.491802   1.454836   5.555664   
4       0  2021-06-21 16:49:00:87975   1.765455   3.118425   2.244614   
...    ..                        ...        ...        ...        ...   
26395  43  2021-06-21 17:10:48:51296   0.957680   0.749753   2.140764   
26396  43    2021-06-21 17:10:48:533   1.959682   0.961444   3.071254   
26397  43  2021-06-21 17:10:48:55273   7.182949   2.138939   8.784543   
26398  43  2021-06-21 17:10:48:57292   0.673608   1.552722   0.112305   
26399  43   2021-06-21 17:10:48:5926   0.491693   0.610908   0.302455   

            gy_x      gy_y      gy_z  acc_Energy  gy_Energy  gy_acc_Energy  \
0       1.335063 -0.137148  0.367154   -0.146398  -0.528656      -0.527245   
1       1.958673  1.669917  1.228946   -0.143188  -0.864807      -0.859051   
2       0.431302  2.745247  0.652966   -0.107894  -0.878751      -0.873028   
3       1.285543  1.648058  4.415028   -0.101785  -0.883770      -0.879247   
4       2.195367  0.140139  4.951928   -0.078672  -0.866098      -0.867841   
...          ...       ...       ...         ...        ...            ...   
26395   3.608055  2.993499  3.142487    0.011410   1.151183       1.144562   
26396   1.240784  0.229501  0.885119    0.220004   0.867653       0.860067   
26397  10.957580  5.098732  9.462141    0.480626   0.616660       0.608726   
26398   3.039773  1.532254  1.456451    0.403118   0.660219       0.652421   
26399   0.818271  0.641538  1.272908    0.494013   0.867212       0.858846   

       acc_x_dt  acc_y_dt  acc_z_dt   gy_x_dt   gy_y_dt   gy_z_dt  \
0     -0.000204 -0.000051  0.000079 -0.000016 -0.000067 -0.000181   
1      0.062211  0.062126  0.027413  0.042070 -1.336735  1.214968   
2     -0.003325 -0.144791  0.000615  0.024655 -0.049573 -0.024484   
3      0.009159  0.038682 -0.023504  0.036265 -0.009496  0.024122   
4      0.021641 -0.087711  0.023661  0.111731  0.004648  0.021084   
...         ...       ...       ...       ...       ...       ...   
26395 -0.329966  0.646183  0.117991 -1.580443  0.127235 -0.638134   
26396  0.176640  0.949933  0.196242 -0.976717  0.044725 -0.142961   
26397  0.343082  0.246619  0.082617 -1.014450 -0.132083  0.221584   
26398 -0.262350 -0.609590  0.281461  0.220576  0.065942 -0.167264   
26399  0.123587  0.236426  0.011334  0.732872 -0.042501  0.248924   

       acc_Energy_dt  gy_Energy_dt  gy_acc_Energy_dt  
0          -0.000085     -0.000602         -0.000603  
1           0.002859     -1.244478         -1.228095  
2           0.032277     -0.052197         -0.052308  
3           0.005517     -0.019175         -0.023610  
4           0.021109      0.064792          0.041594  
...              ...           ...               ...  
26395      -0.198806     -1.894978         -1.893919  
26396       0.191184     -1.049759         -1.053069  
26397       0.238889     -0.929363         -0.930419  
26398      -0.071155      0.160583          0.161042  
26399       0.083260      0.765344          0.763051  

[26400 rows x 20 columns]

##### 모델링

+ CNN, LSTM, CNN+LSTM 등 여러 구조 적용해보다가 CNN에서 Flatten 없이 Global average pooling 한 구조가 가장 성능이 좋아 채택했습니다.


In [8]:
# pip install tensorflow_addons

In [7]:
# pip install np_utils

In [28]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.layers import Dense, LSTM,Bidirectional,Dropout
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras import backend as K 
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
import keras

In [29]:
X=np.array(train_sc.iloc[:,2:]).reshape(468, 600, -1)
X.shape

(468, 600, 18)

In [30]:
test_x=np.array(test_sc.iloc[:,2:]).reshape(44, 600, -1)
# reshape()의 ‘-1’이 의미하는 바는, 변경된 배열의 ‘-1’ 
# 위치의 차원은 “원래 배열의 길이와 남은 차원으로 부터 추정”
# => 차원을 따로 지정하지않고 추정하겠다라는 의미인듯
test_x.shape

(44, 600, 18)

In [31]:
train_labels

id  label label_desc
0      0      1         두손
1      1      1         두손
2      2      1         두손
3      3      1         두손
4      4      1         두손
..   ...    ...        ...
463  463      2         왼손
464  464      2         왼손
465  465      2         왼손
466  466      2         왼손
467  467      2         왼손

[468 rows x 3 columns]

In [32]:
y = train_labels['label'].values
y = tf.keras.utils.to_categorical(train_labels['label']) 
print(y)
print("y.shape => ",y.shape)
y[0]

# y[0] 출력해보면 알겠지만 label 번째 인덱스 값만 1이다(0부터 시작했을 경우)

[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]
y.shape =>  (468, 4)


array([0., 1., 0., 0.], dtype=float32)

##### 모델 구조 

In [33]:
# seed를 지정하면 계속 똑같은 값만 나온다
seed(1)
np.random.rand(10)

array([4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
       1.46755891e-01, 9.23385948e-02, 1.86260211e-01, 3.45560727e-01,
       3.96767474e-01, 5.38816734e-01])

In [34]:
def cnn_model(input_shape, classes):
    # random.seed를 설정해주면 같은 랜덤셋이 나온다했는데..
    # 왜 자꾸 seed를 지정해주는지는 모르겠다 
    seed(2021)
    tf.random.set_seed(2021)
    
    # keras input은 데이터의 입력 모양을 모델에 알려주는 역할
    input_layer = keras.layers.Input(input_shape)
    # Conv1D => 1D 컨볼루션 레이어
    conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)
    conv1 = keras.layers.Dropout(rate=0.3)(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    conv2 = keras.layers.Dropout(rate=0.4)(conv2)
    
    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    conv3 = keras.layers.Dropout(rate=0.5)(conv3)
    
    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    
    output_layer = keras.layers.Dense(classes, activation='softmax')(gap)
    
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
        metrics=['accuracy'])
    
    return model

##### 10-fold StratifiedKFold

In [ ]:
os.getcwd()

'/content/drive/My Drive/data_0625'

In [35]:
# 과적합을 방지하기 위한 교차검증
# StratifiedKFold => target에 속성값 개수를 동일하게 가져감으로써 기존의 kfold와 같은 데이터 몰림현상 방지
skf = StratifiedKFold(n_splits = 10, random_state = 2021, shuffle = True)

# ReduceLROnPlateau => keras의 콜백함수
# 학습률이 개선되지 않을 때 학습률을 동적으로 조정하여 개선하는 효과를 기대할 수 있음
# patience => epoch 4 동안 개선되지 않으면 callback 호출
# verbos => 1일 경우, EarlyStopping이 적용될 때, 화면에 적용되었다고 나타냄, 0일 경우, 화면에 나타냄 없이 종료
# factor => callback 호출 시 학습률을 0.5로 줄인다(1/2)
reLR = ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 

# EarlyStopping => 더 이상 개선의 여지가 없을 때 학습을 종료시키는 콜백함수
# monitor = 'val_loss' => 검증 손실을 기준으로 callback이 호출
# patience => epoch 8 동안 개선되지 않으면 callback 호출
# mode => 관찰 항목이 val_loss인 경우 감소되는 것이 멈출 때 종료되야 하므로 'min'으로 설정
es =EarlyStopping(monitor='val_loss', patience=8, mode='min')

accuracy = []
losss=[]
models=[]

for i, (train, validation) in enumerate(skf.split(X, y.argmax(1))) :
    # ModelCheckpoint => 모델을 저장할 때 사용하는 콜백 함수
    # filepath => 모델을 저장할 경로
    # save_best_only => True 인 경우, monitor 되고 있는 값을 기준으로 가장 좋은 값으로 모델이 저장
    #                => False인 경우, 매 epoch마다 모델이 filepath{epoch}으로 저장 ex) model0,model1,model2..
    # verbose, monitor, mode => 위랑 일맥상통
    # save_weights_only => True인 경우, 모델의 weights만 저장
    #                   -> False인 경우, 모델 레이어 및 weights 모두 저장
    mc = ModelCheckpoint(f'./model_kf/cv_study{i + 1}.h5',save_best_only=True, verbose=0, monitor = 'val_loss', mode = 'min', save_weights_only=True)
    print("-" * 20 +"Fold_"+str(i+1)+ "-" * 20)
    model = cnn_model((600,18),4)
    history = model.fit(X[train], y[train], epochs = 100, validation_data= (X[validation], y[validation]), 
                        verbose=1,batch_size=64,callbacks=[es,mc,reLR])
    model.load_weights(f'./model_kf/cv_study{i + 1}.h5')
    
    # model.evaluate => 테스트 데이터를 통해 학습한 모델에 대한 정확도를 평가
    # 첫번째 인자 = 테스트 데이터에 해당
    # 두번째 인자 = 지도 학습에서 레이블 테스트 데이터에 해당
    # batch_size = 배치 크기
    k_accuracy = '%.4f' % (model.evaluate(X[validation], y[validation])[1])
    k_loss = '%.4f' % (model.evaluate(X[validation], y[validation])[0])
    
    accuracy.append(k_accuracy)
    losss.append(k_loss)
    models.append(model)

print('\nK-fold cross validation Auc: {}'.format(accuracy))
print('\nK-fold cross validation loss: {}'.format(losss))

--------------------Fold_1--------------------
Epoch 1/100
7/7 [==============================] - 47s 178ms/step - loss: 1.2211 - accuracy: 0.4262 - val_loss: 1.4076 - val_accuracy: 0.4894
Epoch 2/100
7/7 [==============================] - 0s 44ms/step - loss: 0.5625 - accuracy: 0.8392 - val_loss: 1.7509 - val_accuracy: 0.4894
Epoch 3/100
7/7 [==============================] - 0s 43ms/step - loss: 0.3561 - accuracy: 0.9327 - val_loss: 1.9849 - val_accuracy: 0.4894
Epoch 4/100
7/7 [==============================] - 0s 44ms/step - loss: 0.2306 - accuracy: 0.9630 - val_loss: 1.8619 - val_accuracy: 0.4681
Epoch 5/100
7/7 [==============================] - 0s 44ms/step - loss: 0.1500 - accuracy: 0.9776 - val_loss: 1.6075 - val_accuracy: 0.4894

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/100
7/7 [==============================] - 0s 44ms/step - loss: 0.1392 - accuracy: 0.9723 - val_loss: 1.0440 - val_accuracy: 0.5532
Epoch 7/100
7/7 [=============

##### 성능 확인 및 제출

In [36]:
print(accuracy)
print()
print(losss)

['1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000']

['0.0289', '0.0059', '0.0188', '0.0042', '0.0121', '0.0206', '0.0046', '0.0242', '0.0142', '0.0123']


In [37]:
print(sum([float(i) for i in accuracy])/10)
print()
print(sum([float(i) for i in losss])/10)

1.0

0.01458


In [38]:
test_X=np.array(test_sc.iloc[:,2:]).reshape(44, 600, -1)
test_X.shape

(44, 600, 18)

In [39]:
preds = []
for model in models:
    pred = model.predict(test_X)
    preds.append(pred)
pred = np.mean(preds, axis=0)
pred

array([[1.00348517e-03, 9.92456079e-01, 1.49307452e-04, 6.39116904e-03],
       [1.30685233e-03, 9.85736012e-01, 1.32567147e-04, 1.28246071e-02],
       [1.81729475e-03, 9.89365876e-01, 3.55832570e-04, 8.46099295e-03],
       [1.20837358e-03, 9.79846835e-01, 8.60649598e-05, 1.88587010e-02],
       [1.65171421e-03, 9.83151913e-01, 8.22762586e-03, 6.96881255e-03],
       [3.00353568e-04, 9.96177077e-01, 8.92556913e-04, 2.62999977e-03],
       [3.55849101e-04, 9.89627004e-01, 6.47572149e-03, 3.54152964e-03],
       [2.74146936e-04, 9.94890809e-01, 3.07487836e-03, 1.76018104e-03],
       [2.90706084e-04, 9.94289398e-01, 2.49135261e-03, 2.92848283e-03],
       [5.54949911e-05, 9.99301910e-01, 4.25777806e-04, 2.16883447e-04],
       [5.45475123e-05, 9.95163739e-01, 4.71077161e-03, 7.10129243e-05],
       [9.31714167e-05, 9.97991264e-01, 1.60968362e-03, 3.05876660e-04],
       [9.97863650e-01, 3.97059048e-04, 2.64795090e-04, 1.47461635e-03],
       [9.98052716e-01, 4.57344198e-04, 2.95189035e

In [40]:
submission=pd.read_csv('./sample_submission.csv')
submission.iloc[:,1:]=pred
submission

id         0         1         2         3
0    0  0.001003  0.992456  0.000149  0.006391
1    1  0.001307  0.985736  0.000133  0.012825
2    2  0.001817  0.989366  0.000356  0.008461
3    3  0.001208  0.979847  0.000086  0.018859
4    4  0.001652  0.983152  0.008228  0.006969
5    5  0.000300  0.996177  0.000893  0.002630
6    6  0.000356  0.989627  0.006476  0.003542
7    7  0.000274  0.994891  0.003075  0.001760
8    8  0.000291  0.994289  0.002491  0.002928
9    9  0.000055  0.999302  0.000426  0.000217
10  10  0.000055  0.995164  0.004711  0.000071
11  11  0.000093  0.997991  0.001610  0.000306
12  12  0.997864  0.000397  0.000265  0.001475
13  13  0.998053  0.000457  0.000295  0.001195
14  14  0.998386  0.000347  0.000284  0.000983
15  15  0.998336  0.000333  0.000263  0.001069
16  16  0.994132  0.002535  0.000469  0.002864
17  17  0.998135  0.000659  0.000305  0.000901
18  18  0.998766  0.000330  0.000229  0.000675
19  19  0.998793  0.000302  0.000238  0.000668
20  20  0.996585  0.000829  0.000637  0.001949
21  21  0.998208  0.000365  0.000524  0.000903
22  22  0.996722  0.000558  0.001018  0.001703
23  23  0.997596  0.000471  0.000736  0.001196
24  24  0.994535  0.001341  0.001144  0.002980
25  25  0.997428  0.000537  0.000771  0.001264
26  26  0.997951  0.000349  0.000620  0.001080
27  27  0.997433  0.000519  0.000784  0.001265
28  28  0.001909  0.019134  0.001851  0.977106
29  29  0.001995  0.012186  0.001350  0.984469
30  30  0.001323  0.019725  0.001085  0.977867
31  31  0.002737  0.026733  0.000582  0.969948
32  32  0.000637  0.005288  0.008420  0.985656
33  33  0.001064  0.030714  0.004370  0.963851
34  34  0.000540  0.006804  0.003079  0.989576
35  35  0.000842  0.008723  0.005842  0.984593
36  36  0.020395  0.496282  0.446706  0.036617
37  37  0.025163  0.122709  0.835750  0.016378
38  38  0.001116  0.105741  0.889438  0.003704
39  39  0.000927  0.052787  0.938314  0.007971
40  40  0.001715  0.092620  0.828020  0.077645
41  41  0.000141  0.008113  0.990230  0.001516
42  42  0.000238  0.015585  0.982016  0.002160
43  43  0.000186  0.017991  0.980346  0.001477

In [41]:
submission.to_csv('./result/0625result.csv',index=False)

In [42]:
test = pd.read_csv('./result/0625result.csv')

In [9]:
# test

In [43]:
df = pd.DataFrame(index=range(0,44), columns=['label', 'max'])

In [44]:
label_list = []
max_list = []
for index,row in test.iterrows(): 
    label_list.append(list(row[1:]).index(max(row[1:])))
    max_list.append(max(row[1:]))
df['label']=label_list
df['max']=max_list
# df

In [ ]:
# exercise = pd.read_csv('./0617.csv')
# exercise=exercise.drop('Time',axis=1)
# exercise=exercise.drop('acc_x',axis=1)
# exercise=exercise.drop('acc_y',axis=1)
# exercise=exercise.drop('acc_z',axis=1)
# exercise=exercise.drop('gy_x',axis=1)
# exercise=exercise.drop('gy_y',axis=1)
# exercise=exercise.drop('gy_z',axis=1)
# exercise = exercise.drop_duplicates()

In [45]:
label = pd.read_csv('./train_labels.csv')
label = label.drop('id',axis=1)
label = label.drop_duplicates()
label = label.sort_values('label')
merge = pd.merge(df,label,on=['label'],how='left')

merge.insert( 0, 'id', None)
merge['id'] = test['id']

exercise = pd.read_csv('./0625test.csv')
exercise.drop(['Time','acc_x','acc_y','acc_z','gy_x','gy_y','gy_z'],axis=1,inplace=True)
exercise.drop_duplicates(inplace=True)

merge = pd.merge(merge,exercise,on='id')
merge
merge.rename(columns = {'label_desc' : '결과'}, inplace = True)
merge.rename(columns = {'exercise' : '실제 운동'}, inplace = True)



In [46]:
merge

id  label       max   결과 실제 운동
0    0      1  0.992456   두손    두손
1    1      1  0.985736   두손    두손
2    2      1  0.989366   두손    두손
3    3      1  0.979847   두손    두손
4    4      1  0.983152   두손    두손
5    5      1  0.996177   두손    두손
6    6      1  0.989627   두손    두손
7    7      1  0.994891   두손    두손
8    8      1  0.994289   두손    두손
9    9      1  0.999302   두손    두손
10  10      1  0.995164   두손    두손
11  11      1  0.997991   두손    두손
12  12      0  0.997864   박수    박수
13  13      0  0.998053   박수    박수
14  14      0  0.998386   박수    박수
15  15      0  0.998336   박수    박수
16  16      0  0.994132   박수    박수
17  17      0  0.998135   박수    박수
18  18      0  0.998766   박수    박수
19  19      0  0.998793   박수    박수
20  20      0  0.996585   박수    박수
21  21      0  0.998208   박수    박수
22  22      0  0.996722   박수    박수
23  23      0  0.997596   박수    박수
24  24      0  0.994535   박수    박수
25  25      0  0.997428   박수    박수
26  26      0  0.997951   박수    박수
27  27      0  0.997433   박수    박수
28  28      3  0.977106  오른손   오른손
29  29      3  0.984469  오른손   오른손
30  30      3  0.977867  오른손   오른손
31  31      3  0.969948  오른손   오른손
32  32      3  0.985656  오른손   오른손
33  33      3  0.963851  오른손   오른손
34  34      3  0.989576  오른손   오른손
35  35      3  0.984593  오른손   오른손
36  36      1  0.496282   두손    왼손
37  37      2  0.835750   왼손    왼손
38  38      2  0.889438   왼손    왼손
39  39      2  0.938314   왼손    왼손
40  40      2  0.828020   왼손    왼손
41  41      2  0.990230   왼손    왼손
42  42      2  0.982016   왼손    왼손
43  43      2  0.980346   왼손    왼손

In [ ]:
merge.to_csv('./result/0625result_mapping.csv',encoding='cp949',index=False)